### 동물상 판정기 ChatGPT 소스 원본 기반 최소한의 수정

In [9]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

# 1. 데이터 로드 및 라벨링
data_dir = r'C:\Data\Sample\Animals2'  # 각 폴더가 들어있는 상위 폴더 경로
model_save_path = 'cnn_model.h5'
img_width, img_height = 150, 150
batch_size = 32
epochs = 50

# 2. 데이터 생성기
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.3)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# 3. CNN 모델 구성
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(len(train_generator.class_indices), activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# 4. 모델 학습
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs
)

# 5. 모델 저장
model.save(model_save_path)

# 6. 새로운 이미지로 예측
def predict_image(image_path, model_path):
    model = load_model(model_path)
    img = load_img(image_path, target_size=(img_width, img_height))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    predictions = model.predict(img_array)
    class_indices = train_generator.class_indices
    class_labels = list(class_indices.keys())
    
    for label, probability in zip(class_labels, predictions[0]):
        print(f"{label}: {probability:.4f}")
    
    return predictions[0]

# 예측 테스트
test_image_path = r'C:\Data\Sample\Beauty.jpg'
predict_image(test_image_path, model_save_path)


Found 38 images belonging to 4 classes.
Found 13 images belonging to 4 classes.
Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 1.3946 - val_accuracy: 0.3077 - val_loss: 5.8654
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 475ms/step - accuracy: 0.2188 - loss: 6.0725 - val_accuracy: 0.2308 - val_loss: 4.0867
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.2308 - val_loss: 4.0867
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.3333 - loss: 2.2241 - val_accuracy: 0.3846 - val_loss: 2.2528
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 485ms/step - accuracy: 0.3438 - loss: 2.4068 - val_accuracy: 0.3846 - val_loss: 1.6081
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.3846 - val_loss: 1.6081
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.4062 - loss: 1.4418 - val_accuracy: 0.3846 - val_loss: 1.3906
Epoch 8/50
1/1 ━━━━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
Cat: 0.8219
Dog: 0.0006
Rabbit: 0.1376
Turtle: 0.0399


array([8.2190198e-01, 5.8953569e-04, 1.3760686e-01, 3.9901633e-02],
      dtype=float32)

### 코드 분리 (학습)

In [14]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

from tkinter import filedialog

data_dir = filedialog.askdirectory(title = 'Training Dataset')

# 1. 데이터 로드 및 라벨링
data_dir = r'C:\Data\Sample\Animals2'  # 각 폴더가 들어있는 상위 폴더 경로
model_save_path = 'cnn_model.h5'
img_width, img_height = 150, 150
batch_size = 32
epochs = 50

# 2. 데이터 생성기
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.3)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# 3. CNN 모델 구성
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(len(train_generator.class_indices), activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# 4. 모델 학습
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs
)

# 5. 모델 저장
model.save(model_save_path)

Found 38 images belonging to 4 classes.
Found 13 images belonging to 4 classes.


c:\Users\550sn\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50


c:\Users\550sn\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.1562 - loss: 1.3923 - val_accuracy: 0.3846 - val_loss: 1.7860
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 417ms/step - accuracy: 0.1667 - loss: 3.5868 - val_accuracy: 0.2308 - val_loss: 2.5310
Epoch 3/50


c:\Users\550sn\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 278ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.2308 - val_loss: 2.5310
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.3125 - loss: 1.8597 - val_accuracy: 0.2308 - val_loss: 1.6458
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 441ms/step - accuracy: 0.0000e+00 - loss: 2.4883 - val_accuracy: 0.2308 - val_loss: 1.3616
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.2308 - val_loss: 1.3616
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.6875 - loss: 1.2091 - val_accuracy: 0.3846 - val_loss: 1.3480
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 359ms/step - accuracy: 0.1667 - loss: 1.3304 - val_accuracy: 0.3846 - val_loss: 1.3184
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.3846 - val_loss: 1.3184
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.8333 - loss: 0.9745 - val_accuracy: 0.3846 - 

### 코드 분리 (이미지 판정)

In [15]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

# 1. 데이터 로드 및 라벨링
model_save_path = 'cnn_model.h5'
img_width, img_height = 150, 150

# 6. 새로운 이미지로 예측
def predict_image(image_path, model_path):
    model = load_model(model_path)
    img = load_img(image_path, target_size=(img_width, img_height))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    predictions = model.predict(img_array)
    class_indices = train_generator.class_indices
    class_labels = list(class_indices.keys())
    
    for label, probability in zip(class_labels, predictions[0]):
        print(f"{label}: {probability:.4f}")
    
    return predictions[0]

# 예측 테스트
from tkinter import filedialog

test_image_path = filedialog.askopenfile(title = 'Target image')
test_image_path = test_image_path.name
predict_image(test_image_path, model_save_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Cat: 0.9418
Dog: 0.0006
Rabbit: 0.0518
Turtle: 0.0058


array([9.4179672e-01, 6.0598011e-04, 5.1770814e-02, 5.8264420e-03],
      dtype=float32)